In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/RecFldGrn


# RecName Settings

In [2]:
import os

rec_folder = 'data/ProcData/RecFolder'

RecName_List = os.listdir(rec_folder)
RecName_List

['PNSect', 'Diag', 'Smoking', 'EC', 'PN', 'A1C', 'PNSectSent', 'P']

In [3]:
import os

RecName = 'P'

# the folder to save the Record.
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(fullrec_folder, '<----- fullrec_folder to save the records')

data/ProcData/RecFolder/P <----- fullrec_folder to save the records


## Load One

In [4]:
from recfldgrn.datapoint import load_df_data_from_folder, DataPoint

####################
RID = 'PID' # Record ID Name, always be PhyID
IDValue = 'P5'   # PhyID value
######################
data_folder = rec_folder
RANGE_SIZE = 10000

# print(data_folder)
Pat = DataPoint(RID, IDValue, data_folder, RANGE_SIZE)
Pat

<PID [P5]>

In [5]:
print(Pat)
dfx = Pat.get_df_rec('P')
dfx

<PID [P5]>


,PID,age,basicInfo
0,P5,73,Female&D


## Load Whole

In [6]:
# import this function
from recfldgrn.datapoint import load_df_data_from_folder
# give the fullrec_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/P


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


# FlGrn: PatEcInfo-InfoGrn

1. RecLevel: 'P' (One and Only One)
2. SynFld: 'BasicInfo'
3. GrnName: 'InfoGrn'
4. Rec2ColList_Dict: {'P': ['age', 'basicInfo']}


In [7]:


###################
RecLevel = 'P'
RecLevelID = f'{RecLevel}ID' # (One and Only One)
SynFld = 'PatEcInfo'
GrnName = 'InfoGrn'

Rec2FldList_Dict = {
    'P': ['age', 'basicInfo'],
    'EC': ['DT_min', 'DT_max'],
} # this should be order.

prefix_ids = []
focal_ids = ['PID']

rec_folder = 'data/ProcData/RecFolder'
###################

In [29]:
import pandas as pd
from functools import reduce

######## we will remove this to graintools.py file
def get_df_whole_from_settings(RecLevel, RecLevelID, 
                               SynFld, GrnName, Rec2FldList_Dict, 
                               prefix_ids, focal_ids, rec_folder
                              ):
    ## get the base IDs in case you need it. # eg: RecLevel: BMI, RecName: P.
    # Way 1
    # df_base = Pat.get_df_rec(RecLevel)[prefix_ids + focal_ids]
    # Way 2
    df_base = load_df_data_from_folder(os.path.join(rec_folder, RecLevel))[prefix_ids + focal_ids]


    L = []
    for RecName, FldList in Rec2FldList_Dict.items():
        #print(RecLevelID, RecName, FldList)
        full_cols = prefix_ids + focal_ids + FldList
        # Way 1
        # df_whole = Pat.get_df_rec(RecName)[full_cols]
        # Way 2
        df_whole = load_df_data_from_folder(os.path.join(rec_folder, RecName))[full_cols]


        # make sure RecLevelID in df_whole.columns
        if RecLevelID not in df_whole.columns:
            on_cols = [i for i in df_base.columns if i in df_whole.columns]
            df_whole = pd.merge(df_base, df_whole, on = on_cols, how = 'left')


        # df_whole_dict[RecName] = df_whole

        df_input_groups = pd.DataFrame([{RecLevelID: RecLevelIDValue, RecName: df_input} 
                                         for RecLevelIDValue, df_input in df_whole.groupby(RecLevelID)])

        L.append(df_input_groups)


    # Merge the dataframes in the list using reduce
    df_whole = reduce(lambda left, right: pd.merge(left, right, on=RecLevelID), L)
    return df_whole


In [30]:
df_whole = get_df_whole_from_settings(RecLevel, RecLevelID, 
                                       SynFld, GrnName, Rec2FldList_Dict, 
                                       prefix_ids, focal_ids, rec_folder
                                      )
df_whole

,PID,P,EC
0,P0,PID age basicInfo 0 P0 52 Male&B,PID DT_min DT_max 0 P0 2023-03-18...
1,P1,PID age basicInfo 1 P1 21 Male&B,PID DT_min DT_max 34 P1 2023-03-18...
2,P2,PID age basicInfo 2 P2 23 Female&B,PID DT_min DT_max 59 P2 2023-03-18...
3,P3,PID age basicInfo 3 P3 77 Female&B,PID DT_min DT_max 85 P3 2023-03-...
4,P4,PID age basicInfo 4 P4 23 Male&A,PID DT_min DT_max 172 P4 2023-03-...
5,P5,PID age basicInfo 5 P5 73 Female&D,PID DT_min DT_max 194 P5 2023-03-...
6,P6,PID age basicInfo 6 P6 34 Female&A,PID DT_min DT_max 217 P6 2023-03-...
7,P7,PID age basicInfo 7 P7 76 Male&C,PID DT_min DT_max 224 P7 2023-03-...


In [31]:
# one sample
df_input = df_whole.iloc[6]
# df_input.to_dict()
df_input['PID']

'P6'

In [32]:
df_input['P']

,PID,age,basicInfo
6,P6,34,Female&A


In [33]:
df_input['EC']

,PID,DT_min,DT_max
217,P6,2023-03-19,2023-04-12
218,P6,2023-03-19,2023-03-28
219,P6,2023-03-19,2023-04-04
220,P6,2023-03-21,2023-04-09
221,P6,2023-03-23,2023-04-04
222,P6,2023-03-25,2023-03-25
223,P6,2023-04-05,2023-04-13


In [34]:
# assert the df_input
assert RecLevelID in df_input
for RecName, FldList in Rec2FldList_Dict.items():
    assert RecName in df_input
    df_recinput = df_input[RecName]
    assert set(FldList).issubset(set(df_recinput.columns))

In [43]:
import itertools
from recfldgrn.graintools import func_convert_Num_factory
import inspect


def grain_str_fn(df_input):
    
    '''
    RecLevel = 'P'
    RecLevelID = f'{RecLevel}ID' # (One and Only One)
    Rec2FldList_Dict = {
        'P': ['age', 'basicInfo'],
        'EC': ['DT_min', 'DT_max'],
    }
    '''
    buffer_dict = {}
    
    # Patient Part: 
    df_recinput = df_input['P'][['age', 'basicInfo']]
    
    # P@Age
    fld = 'age'
    start, end, Min, Max, scale = [18, 80, 1, 120, 5]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = df_recinput[fld].iloc[0]
    li_value_wgt = func_convert_Num_wgt(value)
    li_value_str = [f'{fld}{i}' for i in list(range(1, len(li_value_wgt) + 1))]
    li_value_fld = [fld] * len(li_value_str)
    buffer_dict[fld] = {'li_value_str': li_value_str, 'li_value_wgt': li_value_wgt, 'li_value_fld':li_value_fld}
    
    # P@basicInfo
    fld = 'basicInfo'
    basicInfo_series = df_recinput[fld]# .mean()
    value = basicInfo_series.iloc[0]
    li_value_str = value.split('&')
    li_value_wgt = [1] * len(li_value_str)
    li_value_fld = [fld] * len(li_value_str)
    buffer_dict[fld] = {'li_value_str': li_value_str, 'li_value_wgt': li_value_wgt, 'li_value_fld':li_value_fld}
    
    # EC Part
    df_recinput = df_input['EC'][['DT_min', 'DT_max']]
    
    # P@ECNum
    fld = 'ecNum'
    value = len(df_recinput)
    li_value_str = [f'{fld}{value}' ]
    li_value_wgt = [1] * len(li_value_str)
    li_value_fld = [fld] * len(li_value_str)
    buffer_dict[fld] = {'li_value_str': li_value_str, 'li_value_wgt': li_value_wgt, 'li_value_fld':li_value_fld}
    
    # P@ecDays
    fld = 'ecDays'
    start, end, Min, Max, scale = [0, 100, 0, 500, 5]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = (df_recinput['DT_max'] - df_recinput['DT_min']).mean().total_seconds() / 60 / 60 / 24
    li_value_wgt = func_convert_Num_wgt(value)
    li_value_str = [f'{fld}{i}' for i in list(range(1, len(li_value_wgt) + 1))]
    li_value_fld = [fld] * len(li_value_str)
    buffer_dict[fld] = {'li_value_str': li_value_str, 'li_value_wgt': li_value_wgt, 'li_value_fld':li_value_fld}
    
    # summary
    output = {}
    for i in ['li_value_str', 'li_value_wgt', 'li_value_fld']:
        # output[i] = sum(, [])
        output[i.split('_')[-1]] = list(itertools.chain(*[d[i] for fld, d in buffer_dict.items()]))
        
    return output

grain_str_fn_string = inspect.getsource(grain_str_fn)
print(grain_str_fn_string)


def grain_str_fn(df_input):
    
    '''
    RecLevel = 'P'
    RecLevelID = f'{RecLevel}ID' # (One and Only One)
    Rec2FldList_Dict = {
        'P': ['age', 'basicInfo'],
        'EC': ['DT_min', 'DT_max'],
    }
    '''
    buffer_dict = {}
    
    
    # Patient Part: 
    df_recinput = df_input['P'][['age', 'basicInfo']]
    
    # P@Age
    fld = 'age'
    start, end, Min, Max, scale = [18, 80, 1, 120, 5]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = df_recinput[fld].iloc[0]
    li_value_wgt = func_convert_Num_wgt(value)
    li_value_str = [f'{fld}{i}' for i in list(range(1, len(li_value_wgt) + 1))]
    li_value_fld = [fld] * len(li_value_str)
    buffer_dict[fld] = {'li_value_str': li_value_str, 'li_value_wgt': li_value_wgt, 'li_value_fld':li_value_fld}
    
    # P@basicInfo
    fld = 'basicInfo'
    basicInfo_series = df_recinput[fld]# .mean()
    value = basicInfo_series.iloc[0]
    li_value_str = value.split('&')
    li_value_

In [44]:
output = grain_str_fn(df_input)
for name, grn_list in output.items():
    print('\n')
    print(name)
    print(grn_list)
    print(len(grn_list))



str
['age1', 'age2', 'age3', 'age4', 'age5', 'age6', 'age7', 'age8', 'age9', 'age10', 'age11', 'age12', 'age13', 'age14', 'age15', 'age16', 'age17', 'age18', 'age19', 'Female', 'A', 'ecNum7', 'ecDays1', 'ecDays2', 'ecDays3', 'ecDays4', 'ecDays5', 'ecDays6', 'ecDays7', 'ecDays8', 'ecDays9', 'ecDays10', 'ecDays11', 'ecDays12', 'ecDays13', 'ecDays14', 'ecDays15', 'ecDays16', 'ecDays17', 'ecDays18', 'ecDays19', 'ecDays20', 'ecDays21', 'ecDays22', 'ecDays23', 'ecDays24', 'ecDays25', 'ecDays26', 'ecDays27']
49


wgt
[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1.0, 1.0, 0.5143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
49


fld
['age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'basicInfo', 'basicInfo', 'ecNum', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDa

In [45]:
recfldgrn = f'{RecLevel}@{SynFld}-{GrnName}'
recfldgrn

'P@PatEcInfo-InfoGrn'

In [46]:
df_whole[recfldgrn] = df_whole.apply(lambda df_input: grain_str_fn(df_input), axis = 1)
df_whole

,PID,P,EC,P@PatEcInfo-InfoGrn,P@PatEcInfo-InfoGrn_str,P@PatEcInfo-InfoGrn_wgt,P@PatEcInfo-InfoGrn_fld
0,P0,PID age basicInfo 0 P0 52 Male&B,PID DT_min DT_max 0 P0 2023-03-18...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
1,P1,PID age basicInfo 1 P1 21 Male&B,PID DT_min DT_max 34 P1 2023-03-18...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
2,P2,PID age basicInfo 2 P2 23 Female&B,PID DT_min DT_max 59 P2 2023-03-18...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
3,P3,PID age basicInfo 3 P3 77 Female&B,PID DT_min DT_max 85 P3 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
4,P4,PID age basicInfo 4 P4 23 Male&A,PID DT_min DT_max 172 P4 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
5,P5,PID age basicInfo 5 P5 73 Female&D,PID DT_min DT_max 194 P5 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
6,P6,PID age basicInfo 6 P6 34 Female&A,PID DT_min DT_max 217 P6 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
7,P7,PID age basicInfo 7 P7 76 Male&C,PID DT_min DT_max 224 P7 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."


In [47]:
for i in ['str', 'wgt', 'fld']:
    df_whole[f'{recfldgrn}_{i}'] = df_whole[recfldgrn].apply(lambda x: x[i])
    
df_whole

,PID,P,EC,P@PatEcInfo-InfoGrn,P@PatEcInfo-InfoGrn_str,P@PatEcInfo-InfoGrn_wgt,P@PatEcInfo-InfoGrn_fld
0,P0,PID age basicInfo 0 P0 52 Male&B,PID DT_min DT_max 0 P0 2023-03-18...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
1,P1,PID age basicInfo 1 P1 21 Male&B,PID DT_min DT_max 34 P1 2023-03-18...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
2,P2,PID age basicInfo 2 P2 23 Female&B,PID DT_min DT_max 59 P2 2023-03-18...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
3,P3,PID age basicInfo 3 P3 77 Female&B,PID DT_min DT_max 85 P3 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
4,P4,PID age basicInfo 4 P4 23 Male&A,PID DT_min DT_max 172 P4 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
5,P5,PID age basicInfo 5 P5 73 Female&D,PID DT_min DT_max 194 P5 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
6,P6,PID age basicInfo 6 P6 34 Female&A,PID DT_min DT_max 217 P6 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
7,P7,PID age basicInfo 7 P7 76 Male&C,PID DT_min DT_max 224 P7 2023-03-...,"{'str': ['age1', 'age2', 'age3', 'age4', 'age5...","[age1, age2, age3, age4, age5, age6, age7, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."


# Save FldGrn Info



In [40]:
# df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
# df_FieldGrainInfo

In [19]:
# fldgrn_folder = 'data/ProcData/FldGrnInfo'

# if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

# fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
# df_FieldGrainInfo.to_pickle(fullfldgrn_file)